In [19]:
import pickle
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from nltk.stem import porter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
import os
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn')

In [3]:
corpus = pickle.load(open("pickle_files/corpus.pkl", "rb" ))

In [4]:
cleaned_nouns = pickle.load(open("pickle_files/cleaned_nouns.pkl", "rb" ))

In [6]:
drop_buzz_words = pickle.load(open("pickle_files/drop_buzz_words.pkl", "rb" ))

In [20]:
nouns_verbs_adjs = pickle.load(open("pickle_files/nouns_verbs_adjs.pkl", "rb" ))

In [33]:
Xtreme_clean = pickle.load(open("pickle_files/Xtreme_clean.pkl", "rb" ))

In [34]:
len(Xtreme_clean)

3087727

In [3]:
count_vectorizer2 = CountVectorizer(ngram_range=(1, 4),  
                                   stop_words='english', 
                                   #token_pattern="\\b[a-z][a-z]+\\b",
                                   lowercase=True,
                                   max_df = 0.6)

cv_fit_tran = count_vectorizer2.fit_transform(corpus)

In [35]:
tfidf = TfidfVectorizer(ngram_range=(1, 4),  
                                   stop_words='english', 
                                   #token_pattern="\\b[a-z][a-z]+\\b",
                                   lowercase=True,
                                   max_df = 0.6)
cor_tfidf = tfidf.fit_transform(corpus)

In [36]:
lsa = TruncatedSVD(60, algorithm = 'arpack')
corpus_lsa = lsa.fit_transform(cor_tfidf)
corpus_lsa = Normalizer(copy=False).fit_transform(corpus_lsa)

In [37]:
sum(lsa.explained_variance_ratio_)

0.1561047849801454

In [39]:
terms = tfidf.get_feature_names()

In [40]:
for i, comp in enumerate(lsa.components_):
    Terms_in_Comp = zip(terms,comp)
    sorted_Terms = sorted(Terms_in_Comp, key = lambda x: x[1], reverse=True) [:10]
    print("Topic %d:" %i)
    for term in sorted_Terms:
        print(term[0])
    print(" ")

Topic 0:
trump
trump russia
anti trump
russia
anti
pro trump
pro
trump kim
kim
trump campaign
 
Topic 1:
wa
campaign wa
2016 wa
left wa
team wa
investigation wa
wrong wa
secret wa
tower wa
minutes wa
 
Topic 2:
hi
hi tech
tech
right hi
campaign hi
world hi
women hi
reason hi
hi br
morning hi
 
Topic 3:
said
said th
th
said septemb
said wood
wood
septemb
trump said
said ask
said april
 
Topic 4:
new
cable new
cable
state
washington new
fake new
2018
washington
fake
new busi
 
Topic 5:
state
state run
run
state level
state sponsor
level
state control
sponsor
state fund
control
 
Topic 6:
news
news writ
writ
2018
news ap
news yougov
com
news holli
holli
fake news
 
Topic 7:
function
alabama
presid
trump
said
report
wa
state
hi
time
 
Topic 8:
alabama
alabama februari
alabama bas
republican
alabama republican
februari
bas
democrat
alabama democrat
alabama april
 
Topic 9:
presid
vice presid
vice
ex presid
forever presid
forever
ex
reason presid
scandal presid
virginia presid
 
Topic 10:
20

In [41]:
import os

os.system('say "Your move.....Commander"')

0

In [ ]:
SSEs = []
Sil_coefs = []
for k in range(2,21):
    km = KMeans(n_clusters=k, random_state=1)
    km.fit(corpus_lsa)
    labels = km.labels_
    Sil_coefs.append(silhouette_score(corpus_lsa, labels, metric='euclidean'))
    SSEs.append(km.inertia_) 

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(15,5), sharex=True, dpi=200)
k_clusters = range(2,21)
ax1.plot(k_clusters, Sil_coefs)
ax1.set_xlabel('number of clusters')
ax1.set_ylabel('silhouette coefficient')

# plot here on ax2
ax2.plot(k_clusters, SSEs)
ax2.set_xlabel('number of clusters')
ax2.set_ylabel('SSE');